In [1]:
import importlib
import pandas
import numpy
import re

In [3]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

#### ЗАВАНТАЖЕННЯ ФАЙЛУ З BIPRO.PROZORRO ІЗ ЗАВЕРШЕНИМИ ЗАКУПІВЛЯМИ МІСЬКИХ РАД.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Дата - від 1 серпня 2016 до 31 грудня 2019. 
"ИдентификаторЛота" є унікальним ідентифікатором рядка.¶

In [4]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep=';', low_memory = False)

In [5]:
training_data_columns = df_raw.columns
training_data_columns = training_data_columns.to_list()

#### Крок 0. Створити базу шкіл і садків, які є замовниками за базою МОН (positives)

In [6]:
# Перший спосіб відібрати навчальні заклади, які є замовниками: пошук за ЄДРПОУ в полі "IDOrganizator"
df_independent = filter_dataframe(df_raw, positive_edrpous(), ['IDOrganizator'])
print('df_independent:', df_independent.shape)

# Створити базу для закупівель навчальних закладів, які не є самостійними замовниками
df_merged_left = pandas.merge(df_raw, df_independent, on='ИдентификаторЛота', how='left', indicator=True, suffixes=('','_temp'))
df_undefined = df_merged_left[df_merged_left._merge=='left_only']
df_undefined = df_undefined[training_data_columns]

/home/artem/.local/lib/python3.6/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


537893 rows. After filtering: 38817
df_independent: (38817, 11)


In [7]:
# Другий спосіб відібрати навчальні заклади, які є замовниками: пошук за ключовими словами в полі "Организатор"
step_mark = 'step0'
action_mark = 'check'
number_0 = '0'
columns_to_check_step0 = ['Организатор']
df_step0 = df_undefined.copy()

for column in columns_to_check_step0:
    df_step0 = step_check(df_step0, column, number_0)
    
df_independent_2, df_undefined_2 = separate_positives_and_negatives_ogranizator(df_step0, step_mark, action_mark)
print(f"{step_mark}:", "positives:", df_independent_2.shape, "negatives:", df_undefined_2.shape)

step0: positives: (12598, 12) negatives: (486478, 12)


#### Крок 1. Пошук за ключовими словами закупівель для навчальних закладів, які не є замовниками (negatives)

In [8]:
step_mark = 'step1'
action_mark = 'check'
number_1 = '1'
columns_to_check_step1 = ['Тендер', 'Организатор', 'ОписаниеТендера', 'Лот']
df_step1 = df_undefined_2.copy()

for column in columns_to_check_step1:
    df_step1 = step_check(df_step1, column, number_1)

df_step1_positives, df_step1_negatives = separate_positives_and_negatives(df_step1, step_mark, action_mark)
df_step1_positives = df_step1_positives[training_data_columns]
df_step1_negatives = df_step1_negatives[training_data_columns]
print(f"{step_mark}:", "positives:", df_step1_positives.shape, "negatives:", df_step1_negatives.shape)

step1: positives: (20218, 11) negatives: (466260, 11)


#### Крок 2. За ЄДРПОУ видалити закупівлі навчальних закладів, які не є закладами загальної середньої та дошкільної освіти

In [ ]:
# !!! Управління освіти виключити з переліку
filter_dataframe_inspect_before_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])['Организатор'].unique().tolist()

In [ ]:
filter_dataframe_inspect_before_drop(df_step1_positives, filter_poza, ['Организатор'])['Организатор'].unique().tolist()

In [9]:
df_step2_positives = filter_dataframe_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])
df_step2_positives = filter_dataframe_drop(df_step2_positives, filter_poza, ['Организатор'])

Dropped rows: 593
Dropped rows: 44


#### Крок 3.

In [10]:
frames = [df_independent, df_independent_2, df_step2_positives]
df_step3 = pandas.concat(frames)
print(df_step3.shape)
df_step3.drop_duplicates(subset='ИдентификаторЛота', inplace = True)
print(df_step3.shape)

(70996, 12)
(70614, 12)


#### Крок 4.

In [175]:
step_mark = 'step4'
action_mark = 'classification'
columns_to_classify_step4 = ['Организатор', 'Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step4 = df_step3[training_data_columns].copy()

In [177]:
df_step4['Тип закладу'] = numpy.full((df_step4.shape[0],), filter_inshe)

In [178]:
def step4_classification(given_df, columns_to_classify, filter_list, filter_name):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.contains('|'.join(filter_list), case = False, na = False)
    for i in range(given_df.shape[0]):
        for column in columns_to_classify:
            if given_df.loc[given_df.index[i], column + '_' + step_mark + '_classification'].any():
                given_df.loc[given_df.index[i], 'Тип закладу'] = filter_name
                break
    for column in columns_to_classify:
        given_df = clear_redundant_columns(given_df, step_mark, action_mark, column)
    return given_df

In [179]:
for idx in range(len(filter_priority)):
    df_step4 = step4_classification(df_step4, columns_to_classify_step4, filter_priority[idx], filter_name_priority[idx])

In [186]:
df_step4['Тип закладу'].value_counts()

ДНЗ                                          22182
Інше                                         20144
Загальноосвітня школа                         9368
Спеціалізована школа                          6530
Ліцей                                         5209
Навчально-виховний комплекс (об'єднання)      3075
Гімназія                                      2828
Інтернат                                       962
Спеціальна загальноосвітня школа               304
Міжшкільний навчально-виробничий комбінат       10
Загальносвітня санаторна школа                   2
Name: Тип закладу, dtype: int64